In [6]:
import pandas as pd
import numpy as np
import os
import timeit
import matplotlib.pyplot as plt

pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

### Load Data

In [2]:
data_path = '/Users/jujohnson/cms-data/aggregated/partB-2016-minmax-scaled.hdf5'
with pd.HDFStore(data_path) as hdf:
    print(hdf.keys())

['/partB_raw', '/partB_test_normalized', '/partB_train_normalized']


In [3]:
train_df = pd.HDFStore(data_path).get('partB_train_normalized')
test_df = pd.HDFStore(data_path).get('partB_test_normalized')

In [8]:
train_y = train_df['exclusion']
train_x = train_df.drop(columns='exclusion')

In [9]:
test_y = test_df['exclusion']
test_x = test_df.drop(columns='exclusion')

In [10]:
print(f'Training data shape {train_x.shape}')
print(f'Test data shape {test_x.shape}')

Training data shape (3753896, 125)
Test data shape (938474, 125)


# Chi-Squared Feature Selection

In [52]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import roc_auc_score

In [50]:
feature_selector = SelectKBest(chi2, k=125).fit(train_x, train_y)
train_x_new = feature_selector.transform(train_x)
test_x_new = feature_selector.transform(test_x)

In [54]:
from sklearn.ensemble import RandomForestClassifier
tree_count = 100
rf_model = RandomForestClassifier(n_jobs=6, n_estimators=tree_count)

In [44]:
# take a sample for testing
# sample_size = 100000
# train_x_sample = train_x_new[:sample_size]
# train_y_sample = train_y[:sample_size]

In [58]:
df = pd.DataFrame(train_x_new)

In [61]:
start = timeit.default_timer()

rf_model.fit(train_x_new, train_y)

end = timeit.default_timer()
print(f'Training completed in {end - start} seconds')

Training completed in 712.9987269119993 seconds


In [62]:
posteriors = rf_model.predict_proba(test_x_new)

In [63]:
roc_auc_score(test_y, posteriors[:, 1])

0.6316960214838533